<a href="https://colab.research.google.com/github/korayaslaan/artificial_intelligence/blob/main/dnevnik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
import os
import bs4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import time
import urllib.request
import re
import urllib3
from pandas import DataFrame
import csv
import datetime
from datetime import datetime, timedelta
import concurrent.futures
import os
import csv
import concurrent
import multiprocessing
from multiprocessing import pool
import io
from pprint import pprint

In [114]:
def datecreator():
  t1=datetime(2002,4,3)#en son 2001/02/11 e kadar çalışıyo
  t2=datetime(2002,4,5)
  t = timedelta(days = 1)
  dates = np.arange(t1, t2, t).astype(datetime)
  links=[]
  link ="https://www.dnevnik.bg/allnews/" 
  for i in dates:
    newdate=i.strftime('%Y/%m/%d')
    links.append("{}{}{}".format(link,newdate,'/'))
  return links

In [115]:
def get_link(i):
    r = requests.get(i)   
    soup = BeautifulSoup(r.content, 'html5lib')
    count = 0
    for a in soup.find_all('a', href=True):
        link = a['href']
        count+=1
        result = link.startswith(('/','java','https://www.dnevnik.bg/photos/','#','https://www.dnevnik.bg/print/'),0,30)
        if result == False:
          if len(link)>28:
            # print(link)
            result2 = link.startswith('https://www.dnevnik.bg/')
            if result2 == True:
              result3 = link.endswith(('#comments','kontakti/'))
              if result3 == False and count%2==1:
                with open("dnevnik_links.txt", 'a') as file:
                   file.write(link+'\n')
              

In [116]:
# get_link('https://www.dnevnik.bg/allnews/2008/03/24/')

In [117]:
def creator(url):
    r = requests.get(url)   
    soup = BeautifulSoup(r.content, 'html5lib')
    title = soup.find("h1").getText()
    content_array = soup.find("div", attrs = {"class":"article-content"}).getText()
    # category = soup.find("span", attrs = {"itemprop":"name"}).getText()
    # print(category)
    date = soup.find("time", attrs = {"itemprop":"datePublished"}).getText()
    date = date.split()
    date =date[1] + date[2] +  date[3] 
    title = title.split()
    content_array = content_array.split()
    content_string = ""
    title_string = ""
    for w in title:
      title_string = title_string+" "+w
    for w in content_array:
      content_string = content_string+" "+w
      #w_data = url+";"+date+";"+title+";"+content_string
    w_data="{};{};{};{}".format(url,date,title_string,content_string)
    # write_to_txt(w_data)
    with open("dnevnik_content.txt", 'a') as file: 
        file.write(w_data+'\n')

In [118]:
# creator('https://www.dnevnik.bg/bulgaria/2008/03/24/475416_sudilishta_samo_gledali_da_otlagat_dela/')

In [119]:
links = datecreator()
t1=time.time()
with concurrent.futures.ProcessPoolExecutor() as execut:
  b_res=[execut.submit(get_link,i.strip()) for i in links]

print(time.time()-t1)

1.6833252906799316


In [120]:
dnevnik_links=[]
with open("dnevnik_links.txt",'r',newline='') as f:
  for i in f.readlines():    
    dnevnik_links.append(i)

In [121]:
links = datecreator()

with concurrent.futures.ProcessPoolExecutor() as execut:
  b_res=[execut.submit(creator,i.strip()) for i in dnevnik_links]
